<a href="https://colab.research.google.com/github/yogamaha/cs598-psl/blob/main/Copy_of_PSL_Proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

In [ ]:
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

from google.colab import drive
from datetime import datetime, timedelta

from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge, Lasso,LassoCV,RidgeCV,RidgeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time


import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

TA suggestions:
1. use STOP_WORDS by prof
1. use both train and test to create vocab
2. don't use t-test, just use lasso to reduce vocab to less and 1000
3. try tuning model param for GBM


In [ ]:
STOP_WORDS = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
             'you', 'your', 'yours', 'their', 'they', 'his', 'her', 'she',
             'he', 'a', 'an', 'and', 'is', 'was', 'are', 'were', 'him',
             'himself', 'has', 'have', 'it', 'its', 'the', 'us']


In [ ]:

drive.mount('/content/drive')

train_path ='/content/drive/My Drive/PSL_data/proj3_data/split_1/train.tsv'
test_path = '/content/drive/My Drive/PSL_data/proj3_data/split_1/test.tsv'
test_y_path = '/content/drive/My Drive/PSL_data/proj3_data/split_1/test_y.tsv'

train_df = pd.read_csv(train_path, sep='\t')
test_df = pd.read_csv(test_path, sep='\t')
test_y_df = pd.read_csv(test_y_path, sep='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_whole= pd.concat([test_df, test_y_df['sentiment']], axis=1)

combined_set = train_df[['review','sentiment']].append(test_whole[['review','sentiment']], ignore_index=True)

<ipython-input-24-6c797c933abc>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_set = train_df[['review','sentiment']].append(test_whole[['review','sentiment']], ignore_index=True)


In [ ]:
# clean the html tags.
combined_set['review'] = combined_set['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)

In [ ]:
# step 1. Fit count vectorizer
vectorizer = CountVectorizer(
    preprocessor=lambda x: x.lower(),  # Convert to lowercase
    stop_words=STOP_WORDS,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                        # Minimum document frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b" # Use word tokenizer
)

dtm_train = vectorizer.fit_transform(combined_set['review'])

In [ ]:
dtm_train

<50000x33011 sparse matrix of type '<class 'numpy.int64'>'
	with 8954742 stored elements in Compressed Sparse Row format>

In [ ]:
# Step 2: Use Lasso for feature selection
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(dtm_train)

lasso_model = Lasso(alpha=0.00623)

# Fit the model with L1 regularization
lasso_model.fit(X_train_scaled, combined_set['sentiment'])

# Get the coefficients
lasso_coefs = lasso_model.coef_
print('Lasso Coefficients:', lasso_coefs.sum())

# Create a DataFrame with feature names and their coefficients
feature_coef_df = pd.DataFrame(
    {'Feature': np.array(vectorizer.get_feature_names_out()),
     'Coefficient': lasso_coefs})

"""experiments:
alpha=0.001, vocab: ~10k
alpha=0.005, vocab: 1440
alpha=0.01, vocab: ~400
alpha=0.0067, vocab: 863
alpha=0.0065, vocab: 925
alpha=0.0064, vocab: 942
alpha=0.0063, vocab: 973
alpha=0.0062, vocab: 1002
alpha=0.00625, vocab: 984
alpha=0.00623, vocab: 990
alpha=0.00621, vocab: 998
"""

Lasso Coefficients: -0.22972860162756611


'experiments:\nalpha=0.001, vocab: ~10k\nalpha=0.005, vocab: 1440\nalpha=0.01, vocab: ~400\nalpha=0.0067, vocab: 863\nalpha=0.0065, vocab: 925\nalpha=0.0064, vocab: 942\nalpha=0.0063, vocab: 973\nalpha=0.0062, vocab: 1002\nalpha=0.00625, vocab: 984\nalpha=0.00623, vocab: 990\nalpha=0.00621, vocab: 998\n'

In [ ]:
# step 2.1: filter features.
selected_features = feature_coef_df[feature_coef_df['Coefficient'] != 0]
vocabulary = list(selected_features['Feature'].values)

len(selected_features)

990

In [ ]:

file_path = '/content/drive/My Drive/PSL_data/proj3_data/myvocab.txt'


with open('/content/drive/My Drive/PSL_data/proj3_data/myvocab.txt', 'w') as file:
    # Iterate through the list and write each word to a new line
    for word in vocabulary:
        file.write(word + "\n")


In [ ]:

# Open the file and read its contents into a list
with open('/content/drive/My Drive/PSL_data/proj3_data/myvocab.txt', 'r') as file:
    lines = file.readlines()
vocabulary = [line.strip() for line in lines]

print (len(vocabulary))

990


In [ ]:
fold_ids = [1,2,3,4,5]

train_paths = [
    '/content/drive/My Drive/PSL_data/proj3_data/split_{}/train.tsv'.format(i)
    for i in fold_ids]
test_paths = [
    '/content/drive/My Drive/PSL_data/proj3_data/split_{}/test.tsv'.format(i)
    for i in fold_ids]

test_y_paths = [
    '/content/drive/My Drive/PSL_data/proj3_data/split_{}/test_y.tsv'.format(i)
    for i in fold_ids]


train_dfs = [pd.read_csv(train_path, sep='\t') for train_path in train_paths]
test_dfs = [pd.read_csv(test_path, sep='\t') for test_path in test_paths]

test_y_dfs = [pd.read_csv(testy, sep='\t') for testy in test_y_paths]

In [ ]:
# Step 3: Train model - GradientBoostingRegressor

from sklearn.ensemble import GradientBoostingRegressor

vectorizer = CountVectorizer(
    ngram_range=(1, 2)               # Use 1- to 4-grams
)
vectorizer.fit(vocabulary)


for traindf, testdf, testy in zip(train_dfs, test_dfs, test_y_dfs):

    model_train = vectorizer.transform(traindf['review'])
    model_test = vectorizer.transform(testdf['review'])

    tree_regressor = GradientBoostingRegressor(
    learning_rate=0.02, n_estimators=1000, subsample=1,max_depth=6)

    tree_regressor.fit(model_train, traindf['sentiment'])

    predictions = tree_regressor.predict(model_test)

    auc_score = roc_auc_score(testy['sentiment'], predictions)

    print(f'AUC Score: {auc_score:.3f}')


AUC Score: 0.922
AUC Score: 0.918
AUC Score: 0.921
AUC Score: 0.920
AUC Score: 0.920


In [ ]:
# Step 3: Train model - Ridge regressor
vectorizer = CountVectorizer(
    ngram_range=(1, 4)               # Use 1- to 4-grams
)
vectorizer.fit(vocabulary)


for traindf, testdf, testy in zip(train_dfs, test_dfs, test_y_dfs):
    X_train = vectorizer.transform(traindf['review'])
    X_test = vectorizer.transform(testdf['review'])

    scaler = StandardScaler(with_mean=False)
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    ridge_model = Ridge(alpha = 1)
    ridge_model.fit(X_train, traindf['sentiment'])
    predictions = ridge_model.predict(X_test)


    auc_score = roc_auc_score(testy['sentiment'], predictions)

    print(f'AUC Score: {auc_score:.3f}')




KeyboardInterrupt: ignored

In [ ]:
# Step 3: Train model - LogisticRegression
vectorizer = CountVectorizer(
    ngram_range=(1, 2)               # Use 1- to 4-grams
)
vectorizer.fit(vocabulary)

num_folds = 5

for j in range(1, num_folds + 1):
    start_time = time.time()

    train_path = f"/content/drive/My Drive/PSL_data/proj3_data/split_{j}/train.tsv"
    test_path = f"/content/drive/My Drive/PSL_data/proj3_data/split_{j}/test.tsv"
    test_y_path = f"/content/drive/My Drive/PSL_data/proj3_data/split_{j}/test_y.tsv"

    traindf = pd.read_csv(train_path, sep='\t')
    testdf = pd.read_csv(test_path, sep='\t')
    testy = pd.read_csv(test_y_path, sep='\t')

    traindf['review'] = traindf['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)
    testdf['review'] = testdf['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)


    X_train = vectorizer.transform(traindf['review'])
    X_test = vectorizer.transform(testdf['review'])

    scaler = StandardScaler(with_mean=False)
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    clf = LogisticRegression(penalty='elasticnet',solver='saga',random_state=4844,C=1, l1_ratio=0.2).fit(X_train, traindf['sentiment'])
    predictions = clf.predict_proba(X_test)

    auc_score = roc_auc_score(testy['sentiment'], predictions[:,1])


    print(f'AUC Score for split {j}: {auc_score:.3f} | Execution time : {round(time.time() - start_time, 4)} seconds')

AUC Score for split 1: 0.960 | Execution time : 43.6736 seconds
AUC Score for split 2: 0.960 | Execution time : 45.2871 seconds
AUC Score for split 3: 0.960 | Execution time : 45.2158 seconds
AUC Score for split 4: 0.961 | Execution time : 49.9903 seconds
AUC Score for split 5: 0.960 | Execution time : 43.46 seconds
